In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

  Activating project at `~/Research/PulsarLightcurveExtraction`


In [5]:
using CairoMakie
using CSV
using DataFrames
using FITSIO

Suppose we are fitting a Poisson model to count data in $i = 1, \ldots, N_s$ segments.  Let each segment have a varying background rate, $B_i$, but let the signal be constant across segments, and composed of a linear combination of some basis templates, so that the instantaneous rate when photon $j$ arrives from segment $s_j$ is given by 
$$
\lambda\left( t_j \right) = B_{s_j} + \sum_k M_{jk} A_k,
$$
where the $A_k$ are the amplitudes of the basis templates, and the design matrix, $\mathbf{M}$, has columns given by the basis templates evaluated at the arrival times of the photons.

Recall that the likelihood for a variable-rate Poisson process with observations at times $t_j$ is 
$$
\mathcal{L} = \left[ \prod_j \lambda\left( t_j \right) \right] \exp\left( - \int \mathrm{d} t \, \lambda(t) \right)
$$

Thus the log-likelihood for the set of photons under the Poisson model is given by 
$$
\log \mathcal{L} = -\sum_i T_i B_i - \sum_k \mathcal{M}_k A_k + \sum_j \log\left( B_{s_j} + \sum_k M_{jk} A_k \right),
$$
where $T_i$ is the time of the $i$th segment, and $\mathcal{M}_k$ are the integral of the $k$th basis segments over the observation time.  If, for example, the basis elements of $\mathbf{M}$ are periodic, and the observation time involves many cycles, then a reasonable approximation is that $\mathcal{M}_k \simeq 0$.  We will make this assumption going forward.  

The maximum likelihood conditions are that the derivative with respect to the $B_i$ and $A_k$ is zero:
$$
\frac{\partial \log \mathcal{L}}{\partial B_i} = 0 = -T_i + \sum_{\left\{ j \mid t_j \in T_i \right\}} \frac{1}{B_{i} + \sum_k M_{jk} A_k}
$$
and
$$
\frac{\partial \log \mathcal{L}}{\partial A_k} = 0 = \sum_j \frac{M_{jk}}{B_{s_j} + \sum_k M_{jk} A_k}.
$$

These are not straightforward to solve analytically, but could be solved numerically.  About the maximum likelihood point, the Fisher information is given by the second derivatives of the log-likelihood:
$$
I_{BB} = \frac{\partial^2 \log \mathcal{L}}{\partial B_i \partial B_m} = \delta_{im} \sum_{\left\{ j \mid t_j \in T_i \right\}} \frac{1}{\left( B_{i} + \sum_k M_{jk} A_k \right)^2},
$$
$$
I_{BA} = \frac{\partial^2 \log \mathcal{L}}{\partial B_i \partial A_k} = \sum_{\left\{ j \mid t_j \in T_i \right\}} \frac{M_{jk}}{\left( B_{i} + \sum_k M_{jk} A_k \right)^2},
$$
and
$$
I_{AA} = \frac{\partial^2 \log \mathcal{L}}{\partial A_k \partial A_l} = \sum_{j} \frac{M_{jk} M_{jl}}{\left( B_{s_j} + \sum_k M_{jk} A_k \right)^2}.
$$

...compare to a single segment model, note fits to constant background.  Motivates the following approximation:
